In [1]:
%%time 
import numpy as np
import pandas as pd
import os
import math
import time
import sys
import itertools

pd.options.display.max_rows = 8

path = r'C:\Users\U\Documents\C7_MAS\C7_Project\Boston_Hubway'
os.chdir(path.replace(os.sep, '/'))
os.getcwd()
H1_df = pd.read_csv('hubway_trips.csv')

Wall time: 2.69 s


In [2]:
#period_pre = "2012-08-31T23:00:00"
period_pre = "2012-08-01T00:00:00"
period_beg = "2012-09-01T00:00:00"
period_end = "2012-10-01T00:00:00"

In [3]:
H1_df

,seq_id,hubway_id,status,duration,start_date,strt_statn,end_date,end_statn,bike_nr,subsc_type,zip_code,birth_date,gender
0,1,8,Closed,9,7/28/2011 10:12:00,23.0,7/28/2011 10:12:00,23.0,B00468,Registered,'97217,1976.0,Male
1,2,9,Closed,220,7/28/2011 10:21:00,23.0,7/28/2011 10:25:00,23.0,B00554,Registered,'02215,1966.0,Male
2,3,10,Closed,56,7/28/2011 10:33:00,23.0,7/28/2011 10:34:00,23.0,B00456,Registered,'02108,1943.0,Male
3,4,11,Closed,64,7/28/2011 10:35:00,23.0,7/28/2011 10:36:00,23.0,B00554,Registered,'02116,1981.0,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579021,1579022,1748019,Closed,480,11/30/2013 23:32:00,67.0,11/30/2013 23:40:00,88.0,T01328,Registered,'02143,NaN,Male
1579022,1579023,1748020,Closed,540,11/30/2013 23:32:00,137.0,11/30/2013 23:41:00,133.0,T01310,Casual,NaN,NaN,NaN
1579023,1579024,1748021,Closed,120,11/30/2013 23:36:00,76.0,11/30/2013 23:38:00,68.0,B01478,Registered,'02141,NaN,Male
1579024,1579025,1748022,Closed,420,11/30/2013 23:39:00,68.0,11/30/2013 23:46:00,95.0,T01352,Registered,'02143,NaN,Female


In [4]:
pd.options.display.max_rows = 12
display(H1_df.describe(include='all'))
pd.options.display.max_rows = 8

,seq_id,hubway_id,status,duration,start_date,strt_statn,end_date,end_statn,bike_nr,subsc_type,zip_code,birth_date,gender
count,1.579025e+06,1.579025e+06,1579025,1.579025e+06,1579025,1.579011e+06,1579025,1.578980e+06,1578559,1579025,1106259,350644.000000,1106414
unique,NaN,NaN,1,NaN,521432,NaN,515102,NaN,1163,2,530,NaN,2
top,NaN,NaN,Closed,NaN,9/10/2013 17:04:00,NaN,10/19/2013 16:26:00,NaN,B00490,Registered,'02118,NaN,Male
freq,NaN,NaN,1579025,NaN,25,NaN,27,NaN,2138,1106414,121749,NaN,834714
mean,7.895130e+05,8.865317e+05,NaN,1.200280e+03,NaN,5.438039e+01,NaN,5.425603e+01,NaN,NaN,NaN,1976.295322,NaN
std,4.558254e+05,5.064783e+05,NaN,2.653539e+04,NaN,3.364295e+01,NaN,3.347219e+01,NaN,NaN,NaN,10.999742,NaN
min,1.000000e+00,8.000000e+00,NaN,-6.900000e+03,NaN,3.000000e+00,NaN,3.000000e+00,NaN,NaN,NaN,1932.000000,NaN
25%,3.947570e+05,4.465250e+05,NaN,4.120000e+02,NaN,2.700000e+01,NaN,2.900000e+01,NaN,NaN,NaN,1969.000000,NaN
50%,7.895130e+05,8.950440e+05,NaN,6.600000e+02,NaN,4.800000e+01,NaN,4.800000e+01,NaN,NaN,NaN,1979.000000,NaN
75%,1.184269e+06,1.328083e+06,NaN,1.082000e+03,NaN,7.400000e+01,NaN,7.400000e+01,NaN,NaN,NaN,1985.000000,NaN


In [5]:
H2_df = H1_df.rename(columns = {'duration': 'tripduration', 
                     'start_date': 'starttime',
                    'end_date': 'stoptime',
                    'strt_statn': 'start_station_id',
                    'end_statn': 'end_station_id',
                    'bike_nr': 'bikeid'})\
[['tripduration', 'starttime', 'stoptime', 'start_station_id', 'end_station_id', 'bikeid']]

In [6]:
H2_df

,tripduration,starttime,stoptime,start_station_id,end_station_id,bikeid
0,9,7/28/2011 10:12:00,7/28/2011 10:12:00,23.0,23.0,B00468
1,220,7/28/2011 10:21:00,7/28/2011 10:25:00,23.0,23.0,B00554
2,56,7/28/2011 10:33:00,7/28/2011 10:34:00,23.0,23.0,B00456
3,64,7/28/2011 10:35:00,7/28/2011 10:36:00,23.0,23.0,B00554
...,...,...,...,...,...,...
1579021,480,11/30/2013 23:32:00,11/30/2013 23:40:00,67.0,88.0,T01328
1579022,540,11/30/2013 23:32:00,11/30/2013 23:41:00,137.0,133.0,T01310
1579023,120,11/30/2013 23:36:00,11/30/2013 23:38:00,76.0,68.0,B01478
1579024,420,11/30/2013 23:39:00,11/30/2013 23:46:00,68.0,95.0,T01352


In [7]:
H2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1579025 entries, 0 to 1579024
Data columns (total 6 columns):
tripduration        1579025 non-null int64
starttime           1579025 non-null object
stoptime            1579025 non-null object
start_station_id    1579011 non-null float64
end_station_id      1578980 non-null float64
bikeid              1578559 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 72.3+ MB


In [8]:
H2_df['starttime'] = pd.to_datetime(H2_df['starttime'])

In [9]:
H2_df['stoptime'] = pd.to_datetime(H2_df['stoptime'])

In [10]:
H2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1579025 entries, 0 to 1579024
Data columns (total 6 columns):
tripduration        1579025 non-null int64
starttime           1579025 non-null datetime64[ns]
stoptime            1579025 non-null datetime64[ns]
start_station_id    1579011 non-null float64
end_station_id      1578980 non-null float64
bikeid              1578559 non-null object
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 72.3+ MB


In [11]:

H2_df = H2_df.query(f'"{period_beg}" <= starttime < "{period_end}"')

In [12]:
H2_df.fillna({'start_station_id':-1, 'end_station_id':-1}, inplace = True)

C:\Users\U\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [13]:
H2_df = H2_df.astype({'start_station_id':'int64', 'end_station_id':'int64'})

In [14]:
H2_df

,tripduration,starttime,stoptime,start_station_id,end_station_id,bikeid
465073,2655,2012-09-01 00:01:00,2012-09-01 00:45:00,55,19,B00296
465077,837,2012-09-01 00:02:00,2012-09-01 00:16:00,34,16,B00465
465078,641,2012-09-01 00:05:00,2012-09-01 00:16:00,43,40,T01296
465079,2522,2012-09-01 00:06:00,2012-09-01 00:48:00,60,76,T01285
...,...,...,...,...,...,...
549282,969,2012-09-30 23:52:00,2012-10-01 00:09:00,55,42,T01098
549283,1687,2012-09-30 23:55:00,2012-10-01 00:23:00,22,18,T01113
549284,261,2012-09-30 23:57:00,2012-10-01 00:02:00,55,45,B00468
549285,231,2012-09-30 23:59:00,2012-10-01 00:03:00,55,45,B00587


In [15]:
S1_df = pd.read_csv('hubway_stations.csv')

In [16]:
S1_df

,id,terminal,station,municipal,lat,lng,status
0,3,B32006,Colleges of the Fenway,Boston,42.340021,-71.100812,Existing
1,4,C32000,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616,Existing
2,5,B32012,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179,Existing
3,6,D32000,Cambridge St. at Joy St.,Boston,42.361285,-71.065140,Existing
...,...,...,...,...,...,...,...
138,142,S32009,Packard Ave / Powderhouse Blvd,Somerville,42.404490,-71.123413,Existing
139,143,S32010,Somerville Hospital at Highland Ave / Crocker St,Somerville,42.390820,-71.109420,Existing
140,144,S32011,Teele Square at 239 Holland St,Somerville,42.402763,-71.126908,Existing
141,145,S32012,Summer St at Cutter St,Somerville,42.394002,-71.120406,Existing


In [17]:
S2_df = S1_df[['id', 'lat', 'lng']]

In [18]:
S_Start_df = S2_df.rename(columns={'id':'start_station_id', 'lat':'start_station_latitude', 'lng':'start_station_longitude'})

In [19]:
S_End_df = S2_df.rename(columns={'id':'end_station_id', 'lat':'end_station_latitude', 'lng':'end_station_longitude'})

In [20]:
H3_df = pd.merge(H2_df, S_Start_df, how='left', on = 'start_station_id')

In [21]:
H3_df

,tripduration,starttime,stoptime,start_station_id,end_station_id,bikeid,start_station_latitude,start_station_longitude
0,2655,2012-09-01 00:01:00,2012-09-01 00:45:00,55,19,B00296,42.347265,-71.088088
1,837,2012-09-01 00:02:00,2012-09-01 00:16:00,34,16,B00465,42.346171,-71.099855
2,641,2012-09-01 00:05:00,2012-09-01 00:16:00,43,40,T01296,42.357143,-71.050699
3,2522,2012-09-01 00:06:00,2012-09-01 00:48:00,60,76,T01285,42.360877,-71.071310
...,...,...,...,...,...,...,...,...
84206,969,2012-09-30 23:52:00,2012-10-01 00:09:00,55,42,T01098,42.347265,-71.088088
84207,1687,2012-09-30 23:55:00,2012-10-01 00:23:00,22,18,T01113,42.352175,-71.055547
84208,261,2012-09-30 23:57:00,2012-10-01 00:02:00,55,45,B00468,42.347265,-71.088088
84209,231,2012-09-30 23:59:00,2012-10-01 00:03:00,55,45,B00587,42.347265,-71.088088


In [22]:
H4_df = pd.merge(H3_df, S_End_df, how='left', on = 'end_station_id')

In [23]:
H4_df 

,tripduration,starttime,stoptime,start_station_id,end_station_id,bikeid,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude
0,2655,2012-09-01 00:01:00,2012-09-01 00:45:00,55,19,B00296,42.347265,-71.088088,42.347527,-71.105828
1,837,2012-09-01 00:02:00,2012-09-01 00:16:00,34,16,B00465,42.346171,-71.099855,42.347433,-71.076163
2,641,2012-09-01 00:05:00,2012-09-01 00:16:00,43,40,T01296,42.357143,-71.050699,42.363871,-71.050877
3,2522,2012-09-01 00:06:00,2012-09-01 00:48:00,60,76,T01285,42.360877,-71.071310,42.366426,-71.105495
...,...,...,...,...,...,...,...,...,...,...
84206,969,2012-09-30 23:52:00,2012-10-01 00:09:00,55,42,T01098,42.347265,-71.088088,42.352096,-71.070378
84207,1687,2012-09-30 23:55:00,2012-10-01 00:23:00,22,18,T01113,42.352175,-71.055547,42.363370,-71.129791
84208,261,2012-09-30 23:57:00,2012-10-01 00:02:00,55,45,B00468,42.347265,-71.088088,42.344763,-71.097880
84209,231,2012-09-30 23:59:00,2012-10-01 00:03:00,55,45,B00587,42.347265,-71.088088,42.344763,-71.097880


In [24]:
%%time
SS1_df = pd.read_csv('stationstatus.csv')

Wall time: 17.5 s


In [25]:
SS1_df

,id,station_id,update,nbBikes,nbEmptyDocks,capacity
0,1,3,2011-08-22 16:23:33.557-04,12,5,17
1,2,5,2011-08-22 16:23:33.557-04,4,11,15
2,3,6,2011-08-22 16:23:33.557-04,8,7,15
3,4,8,2011-08-22 16:23:33.557-04,2,13,15
...,...,...,...,...,...,...
30546808,32470761,5,2012-05-14 00:35:46.023-04,8,7,15
30546809,32470762,6,2012-05-14 00:35:46.023-04,12,3,15
30546810,32470763,7,2012-05-14 00:35:46.023-04,0,15,15
30546811,32470764,8,2012-05-14 00:35:46.023-04,13,1,14


In [26]:
SS2_df = SS1_df.rename(columns={'update': 'time', 
                       'nbBikes': 'num_bikes_available',
                      'nbEmptyDocks': 'num_docks_available'})

In [27]:
SS2_df.dtypes

id                      int64
station_id              int64
time                   object
num_bikes_available     int64
num_docks_available     int64
capacity                int64
dtype: object

In [28]:
%%time
SS2_df['time'] = pd.to_datetime(SS2_df['time'])

Wall time: 4min 40s


In [29]:

SS3_df = SS2_df.query(f'"{period_pre}" <= time < "{period_end}"')

In [30]:
SS3_df

,id,station_id,time,num_bikes_available,num_docks_available,capacity
8542106,8542512,3,2012-08-01 19:08:59.504,12,3,15
8542107,8542513,4,2012-08-01 19:08:59.504,4,11,15
8542108,8542514,5,2012-08-01 19:08:59.504,9,5,14
8542109,8542515,6,2012-08-01 19:08:59.504,2,12,14
...,...,...,...,...,...,...
16644512,16739881,95,2012-09-30 23:59:55.968,13,2,15
16644513,16739882,96,2012-09-30 23:59:55.968,7,12,19
16644514,16739883,97,2012-09-30 23:59:55.968,9,7,16
16644515,16739884,98,2012-09-30 23:59:55.968,9,9,18


In [31]:
%%time
SS3_df = SS3_df.drop(columns=['id'])

Wall time: 152 ms


In [32]:
%%time
SS3_df = SS3_df.sort_values(by=['station_id', 'time'])

Wall time: 4.55 s


In [33]:
SS3_df

,station_id,time,num_bikes_available,num_docks_available,capacity
9045416,3,2012-08-01 00:00:07.511,7,8,15
9045573,3,2012-08-01 00:01:07.960,7,8,15
9045730,3,2012-08-01 00:02:08.404,7,8,15
9045887,3,2012-08-01 00:03:08.827,7,8,15
...,...,...,...,...,...
16644230,98,2012-09-30 23:56:54.723,10,8,18
16644325,98,2012-09-30 23:57:55.164,10,8,18
16644420,98,2012-09-30 23:58:55.566,9,9,18
16644515,98,2012-09-30 23:59:55.968,9,9,18


In [34]:
%%time
SS3_df['time'] = pd.to_datetime(SS3_df['time'].dt.strftime("%Y-%m-%dT%H:00:00"))

Wall time: 43.6 s


In [35]:
SS4_df = SS3_df.drop_duplicates(subset = ['station_id', 'time'], keep='last')

In [36]:
SS4_df.reset_index(inplace=True, drop=True)

In [37]:
SS4_df

,station_id,time,num_bikes_available,num_docks_available,capacity
0,3,2012-08-01 00:00:00,7,8,15
1,3,2012-08-01 01:00:00,7,8,15
2,3,2012-08-01 02:00:00,8,7,15
3,3,2012-08-01 03:00:00,8,7,15
...,...,...,...,...,...
126003,98,2012-09-30 20:00:00,0,18,18
126004,98,2012-09-30 21:00:00,0,18,18
126005,98,2012-09-30 22:00:00,7,11,18
126006,98,2012-09-30 23:00:00,9,9,18


In [38]:
SS4_1_df = SS4_df.copy()

In [39]:
SS4_1_df['time'] = SS4_1_df['time'] + pd.DateOffset(hours=1)

In [40]:
SS4_1_df

,station_id,time,num_bikes_available,num_docks_available,capacity
0,3,2012-08-01 01:00:00,7,8,15
1,3,2012-08-01 02:00:00,7,8,15
2,3,2012-08-01 03:00:00,8,7,15
3,3,2012-08-01 04:00:00,8,7,15
...,...,...,...,...,...
126003,98,2012-09-30 21:00:00,0,18,18
126004,98,2012-09-30 22:00:00,0,18,18
126005,98,2012-09-30 23:00:00,7,11,18
126006,98,2012-10-01 00:00:00,9,9,18


In [41]:
SS5_df = pd.merge(SS4_df, SS4_1_df, how='left', on=['station_id', 'time'], suffixes=('_end', ''))

In [42]:
#SS5_df.dropna(inplace=True)
SS5_df = SS5_df.query(f'time >= "{period_beg}"')

In [43]:
SS5_df = SS5_df.ffill()

In [44]:

SS5_df.set_index(['station_id', 'time'], inplace=True, drop=True)
SS5_df = SS5_df.astype('int64')

In [45]:
SS5_df

num_bikes_available_end  \
station_id time                                           
3          2012-09-01 00:00:00                        5   
           2012-09-01 01:00:00                        2   
           2012-09-01 02:00:00                        2   
           2012-09-01 03:00:00                        3   
...                                                 ...   
98         2012-09-30 20:00:00                        0   
           2012-09-30 21:00:00                        0   
           2012-09-30 22:00:00                        7   
           2012-09-30 23:00:00                        9   

                                num_docks_available_end  capacity_end  \
station_id time                                                         
3          2012-09-01 00:00:00                       10            15   
           2012-09-01 01:00:00                       13            15   
           2012-09-01 02:00:00                       13            15   
           2012-09-01 03:00:00                       12            15   
...                                                 ...           ...   
98         2012-09-30 20:00:00                       18            18   
           2012-09-30 21:00:00                       18            18   
           2012-09-30 22:00:00                       11            18   
           2012-09-30 23:00:00                        9            18   

                                num_bikes_available  num_docks_available  \
station_id time                                                            
3          2012-09-01 00:00:00                    5                   10   
           2012-09-01 01:00:00                    5                   10   
           2012-09-01 02:00:00                    2                   13   
           2012-09-01 03:00:00                    2                   13   
...                                             ...                  ...   
98         2012-09-30 20:00:00                    4                   14   
           2012-09-30 21:00:00                    0                   18   
           2012-09-30 22:00:00                    0                   18   
           2012-09-30 23:00:00                    7                   11   

                                capacity  
station_id time                           
3          2012-09-01 00:00:00        15  
           2012-09-01 01:00:00        15  
           2012-09-01 02:00:00        15  
           2012-09-01 03:00:00        15  
...                                  ...  
98         2012-09-30 20:00:00        18  
           2012-09-30 21:00:00        18  
           2012-09-30 22:00:00        18  
           2012-09-30 23:00:00        18  

[66469 rows x 6 columns]

In [46]:
SS5_df['bike_flow_count'] = SS5_df[['num_bikes_available', 'num_bikes_available_end']].apply(lambda x: x[1] - x[0], axis=1)

In [47]:
SS5_df

num_bikes_available_end  \
station_id time                                           
3          2012-09-01 00:00:00                        5   
           2012-09-01 01:00:00                        2   
           2012-09-01 02:00:00                        2   
           2012-09-01 03:00:00                        3   
...                                                 ...   
98         2012-09-30 20:00:00                        0   
           2012-09-30 21:00:00                        0   
           2012-09-30 22:00:00                        7   
           2012-09-30 23:00:00                        9   

                                num_docks_available_end  capacity_end  \
station_id time                                                         
3          2012-09-01 00:00:00                       10            15   
           2012-09-01 01:00:00                       13            15   
           2012-09-01 02:00:00                       13            15   
           2012-09-01 03:00:00                       12            15   
...                                                 ...           ...   
98         2012-09-30 20:00:00                       18            18   
           2012-09-30 21:00:00                       18            18   
           2012-09-30 22:00:00                       11            18   
           2012-09-30 23:00:00                        9            18   

                                num_bikes_available  num_docks_available  \
station_id time                                                            
3          2012-09-01 00:00:00                    5                   10   
           2012-09-01 01:00:00                    5                   10   
           2012-09-01 02:00:00                    2                   13   
           2012-09-01 03:00:00                    2                   13   
...                                             ...                  ...   
98         2012-09-30 20:00:00                    4                   14   
           2012-09-30 21:00:00                    0                   18   
           2012-09-30 22:00:00                    0                   18   
           2012-09-30 23:00:00                    7                   11   

                                capacity  bike_flow_count  
station_id time                                            
3          2012-09-01 00:00:00        15                0  
           2012-09-01 01:00:00        15               -3  
           2012-09-01 02:00:00        15                0  
           2012-09-01 03:00:00        15                1  
...                                  ...              ...  
98         2012-09-30 20:00:00        18               -4  
           2012-09-30 21:00:00        18                0  
           2012-09-30 22:00:00        18                7  
           2012-09-30 23:00:00        18                2  

[66469 rows x 7 columns]

In [48]:
cols = [c for c in SS5_df.columns if c.lower()[-4:] != '_end']
cols

['num_bikes_available', 'num_docks_available', 'capacity', 'bike_flow_count']

In [49]:
SS5_df = SS5_df[cols]

In [50]:
SS5_df

num_bikes_available  num_docks_available  \
station_id time                                                            
3          2012-09-01 00:00:00                    5                   10   
           2012-09-01 01:00:00                    5                   10   
           2012-09-01 02:00:00                    2                   13   
           2012-09-01 03:00:00                    2                   13   
...                                             ...                  ...   
98         2012-09-30 20:00:00                    4                   14   
           2012-09-30 21:00:00                    0                   18   
           2012-09-30 22:00:00                    0                   18   
           2012-09-30 23:00:00                    7                   11   

                                capacity  bike_flow_count  
station_id time                                            
3          2012-09-01 00:00:00        15                0  
           2012-09-01 01:00:00        15               -3  
           2012-09-01 02:00:00        15                0  
           2012-09-01 03:00:00        15                1  
...                                  ...              ...  
98         2012-09-30 20:00:00        18               -4  
           2012-09-30 21:00:00        18                0  
           2012-09-30 22:00:00        18                7  
           2012-09-30 23:00:00        18                2  

[66469 rows x 4 columns]

In [51]:
_time_df = pd.DataFrame(pd.date_range(start = '2012-09-01T00:00:00', end = '2012-10-01T00:00:00', closed='left', freq='h'), columns=['time']).reset_index(drop=True)
_time_df

,time
0,2012-09-01 00:00:00
1,2012-09-01 01:00:00
2,2012-09-01 02:00:00
3,2012-09-01 03:00:00
...,...
716,2012-09-30 20:00:00
717,2012-09-30 21:00:00
718,2012-09-30 22:00:00
719,2012-09-30 23:00:00


In [52]:
_station_df = S2_df.rename(columns={'id':'station_id', 'lat':'station_latitude', 'lng':'station_longitude'})

In [53]:
_station_df

,station_id,station_latitude,station_longitude
0,3,42.340021,-71.100812
1,4,42.345392,-71.069616
2,5,42.341814,-71.090179
3,6,42.361285,-71.065140
...,...,...,...
138,142,42.404490,-71.123413
139,143,42.390820,-71.109420
140,144,42.402763,-71.126908
141,145,42.394002,-71.120406


In [54]:
_station_df['key'] = 0
_time_df['key'] = 0

_station_time_df = pd.merge(_station_df, _time_df, on='key').drop('key', axis=1)

In [55]:
_station_time_list = _station_time_df.columns.tolist()

In [56]:
_station_time_df.set_index(_station_time_list, inplace=True)

In [57]:
_station_time_df

Empty DataFrame
Columns: []
Index: [(3, 42.340021, -71.100812, 2012-09-01 00:00:00), (3, 42.340021, -71.100812, 2012-09-01 01:00:00), (3, 42.340021, -71.100812, 2012-09-01 02:00:00), (3, 42.340021, -71.100812, 2012-09-01 03:00:00), (3, 42.340021, -71.100812, 2012-09-01 04:00:00), (3, 42.340021, -71.100812, 2012-09-01 05:00:00), (3, 42.340021, -71.100812, 2012-09-01 06:00:00), (3, 42.340021, -71.100812, 2012-09-01 07:00:00), (3, 42.340021, -71.100812, 2012-09-01 08:00:00), (3, 42.340021, -71.100812, 2012-09-01 09:00:00), (3, 42.340021, -71.100812, 2012-09-01 10:00:00), (3, 42.340021, -71.100812, 2012-09-01 11:00:00), (3, 42.340021, -71.100812, 2012-09-01 12:00:00), (3, 42.340021, -71.100812, 2012-09-01 13:00:00), (3, 42.340021, -71.100812, 2012-09-01 14:00:00), (3, 42.340021, -71.100812, 2012-09-01 15:00:00), (3, 42.340021, -71.100812, 2012-09-01 16:00:00), (3, 42.340021, -71.100812, 2012-09-01 17:00:00), (3, 42.340021, -71.100812, 2012-09-01 18:00:00), (3, 42.340021, -71.100812, 2012-09-01 19:00:00), (3, 42.340021, -71.100812, 2012-09-01 20:00:00), (3, 42.340021, -71.100812, 2012-09-01 21:00:00), (3, 42.340021, -71.100812, 2012-09-01 22:00:00), (3, 42.340021, -71.100812, 2012-09-01 23:00:00), (3, 42.340021, -71.100812, 2012-09-02 00:00:00), (3, 42.340021, -71.100812, 2012-09-02 01:00:00), (3, 42.340021, -71.100812, 2012-09-02 02:00:00), (3, 42.340021, -71.100812, 2012-09-02 03:00:00), (3, 42.340021, -71.100812, 2012-09-02 04:00:00), (3, 42.340021, -71.100812, 2012-09-02 05:00:00), (3, 42.340021, -71.100812, 2012-09-02 06:00:00), (3, 42.340021, -71.100812, 2012-09-02 07:00:00), (3, 42.340021, -71.100812, 2012-09-02 08:00:00), (3, 42.340021, -71.100812, 2012-09-02 09:00:00), (3, 42.340021, -71.100812, 2012-09-02 10:00:00), (3, 42.340021, -71.100812, 2012-09-02 11:00:00), (3, 42.340021, -71.100812, 2012-09-02 12:00:00), (3, 42.340021, -71.100812, 2012-09-02 13:00:00), (3, 42.340021, -71.100812, 2012-09-02 14:00:00), (3, 42.340021, -71.100812, 2012-09-02 15:00:00), (3, 42.340021, -71.100812, 2012-09-02 16:00:00), (3, 42.340021, -71.100812, 2012-09-02 17:00:00), (3, 42.340021, -71.100812, 2012-09-02 18:00:00), (3, 42.340021, -71.100812, 2012-09-02 19:00:00), (3, 42.340021, -71.100812, 2012-09-02 20:00:00), (3, 42.340021, -71.100812, 2012-09-02 21:00:00), (3, 42.340021, -71.100812, 2012-09-02 22:00:00), (3, 42.340021, -71.100812, 2012-09-02 23:00:00), (3, 42.340021, -71.100812, 2012-09-03 00:00:00), (3, 42.340021, -71.100812, 2012-09-03 01:00:00), (3, 42.340021, -71.100812, 2012-09-03 02:00:00), (3, 42.340021, -71.100812, 2012-09-03 03:00:00), (3, 42.340021, -71.100812, 2012-09-03 04:00:00), (3, 42.340021, -71.100812, 2012-09-03 05:00:00), (3, 42.340021, -71.100812, 2012-09-03 06:00:00), (3, 42.340021, -71.100812, 2012-09-03 07:00:00), (3, 42.340021, -71.100812, 2012-09-03 08:00:00), (3, 42.340021, -71.100812, 2012-09-03 09:00:00), (3, 42.340021, -71.100812, 2012-09-03 10:00:00), (3, 42.340021, -71.100812, 2012-09-03 11:00:00), (3, 42.340021, -71.100812, 2012-09-03 12:00:00), (3, 42.340021, -71.100812, 2012-09-03 13:00:00), (3, 42.340021, -71.100812, 2012-09-03 14:00:00), (3, 42.340021, -71.100812, 2012-09-03 15:00:00), (3, 42.340021, -71.100812, 2012-09-03 16:00:00), (3, 42.340021, -71.100812, 2012-09-03 17:00:00), (3, 42.340021, -71.100812, 2012-09-03 18:00:00), (3, 42.340021, -71.100812, 2012-09-03 19:00:00), (3, 42.340021, -71.100812, 2012-09-03 20:00:00), (3, 42.340021, -71.100812, 2012-09-03 21:00:00), (3, 42.340021, -71.100812, 2012-09-03 22:00:00), (3, 42.340021, -71.100812, 2012-09-03 23:00:00), (3, 42.340021, -71.100812, 2012-09-04 00:00:00), (3, 42.340021, -71.100812, 2012-09-04 01:00:00), (3, 42.340021, -71.100812, 2012-09-04 02:00:00), (3, 42.340021, -71.100812, 2012-09-04 03:00:00), (3, 42.340021, -71.100812, 2012-09-04 04:00:00), (3, 42.340021, -71.100812, 2012-09-04 05:00:00), (3, 42.340021, -71.100812, 2012-09-04 06:00:00), (3, 42.340021, -71.100812, 2012-09-04 07:00:00), (3, 42.340021, -71.100812, 2012-09-04 08:00:

In [58]:
H4_df 

,tripduration,starttime,stoptime,start_station_id,end_station_id,bikeid,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude
0,2655,2012-09-01 00:01:00,2012-09-01 00:45:00,55,19,B00296,42.347265,-71.088088,42.347527,-71.105828
1,837,2012-09-01 00:02:00,2012-09-01 00:16:00,34,16,B00465,42.346171,-71.099855,42.347433,-71.076163
2,641,2012-09-01 00:05:00,2012-09-01 00:16:00,43,40,T01296,42.357143,-71.050699,42.363871,-71.050877
3,2522,2012-09-01 00:06:00,2012-09-01 00:48:00,60,76,T01285,42.360877,-71.071310,42.366426,-71.105495
...,...,...,...,...,...,...,...,...,...,...
84206,969,2012-09-30 23:52:00,2012-10-01 00:09:00,55,42,T01098,42.347265,-71.088088,42.352096,-71.070378
84207,1687,2012-09-30 23:55:00,2012-10-01 00:23:00,22,18,T01113,42.352175,-71.055547,42.363370,-71.129791
84208,261,2012-09-30 23:57:00,2012-10-01 00:02:00,55,45,B00468,42.347265,-71.088088,42.344763,-71.097880
84209,231,2012-09-30 23:59:00,2012-10-01 00:03:00,55,45,B00587,42.347265,-71.088088,42.344763,-71.097880


In [59]:
H5_df = H4_df.copy()

In [60]:
H5_df['starttime'] = pd.to_datetime(H5_df['starttime'].dt.strftime("%Y-%m-%dT%H:00:00"))

In [61]:
H5_df['stoptime'] = pd.to_datetime(H5_df['stoptime'].dt.strftime("%Y-%m-%dT%H:00:00"))

In [62]:

_6_1_df = H5_df.query(f'"{period_beg}" <= starttime < "{period_end}"')

In [63]:
_6_1_df

,tripduration,starttime,stoptime,start_station_id,end_station_id,bikeid,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude
0,2655,2012-09-01 00:00:00,2012-09-01,55,19,B00296,42.347265,-71.088088,42.347527,-71.105828
1,837,2012-09-01 00:00:00,2012-09-01,34,16,B00465,42.346171,-71.099855,42.347433,-71.076163
2,641,2012-09-01 00:00:00,2012-09-01,43,40,T01296,42.357143,-71.050699,42.363871,-71.050877
3,2522,2012-09-01 00:00:00,2012-09-01,60,76,T01285,42.360877,-71.071310,42.366426,-71.105495
...,...,...,...,...,...,...,...,...,...,...
84206,969,2012-09-30 23:00:00,2012-10-01,55,42,T01098,42.347265,-71.088088,42.352096,-71.070378
84207,1687,2012-09-30 23:00:00,2012-10-01,22,18,T01113,42.352175,-71.055547,42.363370,-71.129791
84208,261,2012-09-30 23:00:00,2012-10-01,55,45,B00468,42.347265,-71.088088,42.344763,-71.097880
84209,231,2012-09-30 23:00:00,2012-10-01,55,45,B00587,42.347265,-71.088088,42.344763,-71.097880


In [64]:
station_list = ['station_id', 'station_latitude', 'station_longitude']

In [65]:
_departure_count_df = \
_6_1_df\
.groupby(['start_station_id', 'start_station_latitude', 'start_station_longitude', 'starttime'], as_index=True)[['bikeid']]\
.count().rename(columns={'bikeid':'departure_count'})
_departure_count_df.index.names = station_list + ['time']

In [66]:
_departure_count_df

departure_count
station_id station_latitude station_longitude time                                
3          42.340021        -71.100812        2012-09-01 08:00:00                1
                                              2012-09-01 09:00:00                2
                                              2012-09-01 11:00:00                1
                                              2012-09-01 13:00:00                4
...                                                                            ...
98         42.371848        -71.060292        2012-09-30 12:00:00                2
                                              2012-09-30 16:00:00                4
                                              2012-09-30 19:00:00                1
                                              2012-09-30 20:00:00                1

[30123 rows x 1 columns]

In [67]:
_departure_count_df['departure_count'].sum()

84198

In [68]:
_arrival_count_df = \
_6_1_df\
.groupby(['end_station_id', 'end_station_latitude', 'end_station_longitude', 'stoptime'], as_index=True)[['bikeid']]\
.count().rename(columns={'bikeid':'arrival_count'})
_arrival_count_df.index.names = station_list + ['time']

In [69]:
_arrival_count_df 

arrival_count
station_id station_latitude station_longitude time                              
3          42.340021        -71.100812        2012-09-01 02:00:00              1
                                              2012-09-01 04:00:00              1
                                              2012-09-01 08:00:00              2
                                              2012-09-01 12:00:00              1
...                                                                          ...
98         42.371848        -71.060292        2012-09-30 12:00:00              3
                                              2012-09-30 14:00:00              1
                                              2012-09-30 19:00:00              2
                                              2012-09-30 21:00:00              2

[30274 rows x 1 columns]

In [70]:
_arrival_count_df['arrival_count'].sum()

84192

In [71]:
from functools import reduce

In [72]:
dfs = [_station_time_df, _departure_count_df, _arrival_count_df]

In [73]:
_7_df = reduce(lambda left,right: pd.merge(left, right, how = 'outer', left_index=True, right_index=True), dfs)

In [74]:
_7_df.fillna(0, inplace=True)

In [75]:
_7_df = _7_df.astype('int64')

In [76]:
_7_df

departure_count  \
station_id station_latitude station_longitude time                                   
3          42.340021        -71.100812        2012-09-01 00:00:00                0   
                                              2012-09-01 01:00:00                0   
                                              2012-09-01 02:00:00                0   
                                              2012-09-01 03:00:00                0   
...                                                                            ...   
145        42.394002        -71.120406        2012-09-30 20:00:00                0   
                                              2012-09-30 21:00:00                0   
                                              2012-09-30 22:00:00                0   
                                              2012-09-30 23:00:00                0   

                                                                   arrival_count  
station_id station_latitude station_longitude time                                
3          42.340021        -71.100812        2012-09-01 00:00:00              0  
                                              2012-09-01 01:00:00              0  
                                              2012-09-01 02:00:00              1  
                                              2012-09-01 03:00:00              0  
...                                                                          ...  
145        42.394002        -71.120406        2012-09-30 20:00:00              0  
                                              2012-09-30 21:00:00              0  
                                              2012-09-30 22:00:00              0  
                                              2012-09-30 23:00:00              0  

[102248 rows x 2 columns]

In [77]:
%%time
_7_df['trip_flow_count'] = _7_df.apply(lambda x: -x[0] +x[1], axis=1)

Wall time: 6.59 s


In [78]:
_7_df

departure_count  \
station_id station_latitude station_longitude time                                   
3          42.340021        -71.100812        2012-09-01 00:00:00                0   
                                              2012-09-01 01:00:00                0   
                                              2012-09-01 02:00:00                0   
                                              2012-09-01 03:00:00                0   
...                                                                            ...   
145        42.394002        -71.120406        2012-09-30 20:00:00                0   
                                              2012-09-30 21:00:00                0   
                                              2012-09-30 22:00:00                0   
                                              2012-09-30 23:00:00                0   

                                                                   arrival_count  \
station_id station_latitude station_longitude time                                 
3          42.340021        -71.100812        2012-09-01 00:00:00              0   
                                              2012-09-01 01:00:00              0   
                                              2012-09-01 02:00:00              1   
                                              2012-09-01 03:00:00              0   
...                                                                          ...   
145        42.394002        -71.120406        2012-09-30 20:00:00              0   
                                              2012-09-30 21:00:00              0   
                                              2012-09-30 22:00:00              0   
                                              2012-09-30 23:00:00              0   

                                                                   trip_flow_count  
station_id station_latitude station_longitude time                                  
3          42.340021        -71.100812        2012-09-01 00:00:00                0  
                                              2012-09-01 01:00:00                0  
                                              2012-09-01 02:00:00                1  
                                              2012-09-01 03:00:00                0  
...                                                                            ...  
145        42.394002        -71.120406        2012-09-30 20:00:00                0  
                                              2012-09-30 21:00:00                0  
                                              2012-09-30 22:00:00                0  
                                              2012-09-30 23:00:00                0  

[102248 rows x 3 columns]

In [79]:
_7_df.sum()

departure_count    84198
arrival_count      84192
trip_flow_count       -6
dtype: int64

In [80]:
H8_df = _7_df.copy()

In [81]:
H8_df.reset_index(inplace=True)

In [82]:
H8_df.set_index(['station_id', 'time'], inplace=True)

In [83]:
H8_df

station_latitude  station_longitude  \
station_id time                                                       
3          2012-09-01 00:00:00         42.340021         -71.100812   
           2012-09-01 01:00:00         42.340021         -71.100812   
           2012-09-01 02:00:00         42.340021         -71.100812   
           2012-09-01 03:00:00         42.340021         -71.100812   
...                                          ...                ...   
145        2012-09-30 20:00:00         42.394002         -71.120406   
           2012-09-30 21:00:00         42.394002         -71.120406   
           2012-09-30 22:00:00         42.394002         -71.120406   
           2012-09-30 23:00:00         42.394002         -71.120406   

                                departure_count  arrival_count  \
station_id time                                                  
3          2012-09-01 00:00:00                0              0   
           2012-09-01 01:00:00                0              0   
           2012-09-01 02:00:00                0              1   
           2012-09-01 03:00:00                0              0   
...                                         ...            ...   
145        2012-09-30 20:00:00                0              0   
           2012-09-30 21:00:00                0              0   
           2012-09-30 22:00:00                0              0   
           2012-09-30 23:00:00                0              0   

                                trip_flow_count  
station_id time                                  
3          2012-09-01 00:00:00                0  
           2012-09-01 01:00:00                0  
           2012-09-01 02:00:00                1  
           2012-09-01 03:00:00                0  
...                                         ...  
145        2012-09-30 20:00:00                0  
           2012-09-30 21:00:00                0  
           2012-09-30 22:00:00                0  
           2012-09-30 23:00:00                0  

[102248 rows x 5 columns]

In [84]:
H9_df = pd.merge(H8_df, SS5_df, how='left', left_index=True, right_index=True)

In [85]:
H9_df.reset_index(inplace=True)


In [86]:
H9_df.count()

station_id             102248
time                   102248
station_latitude       102248
station_longitude      102248
                        ...  
num_bikes_available     66469
num_docks_available     66469
capacity                66469
bike_flow_count         66469
Length: 11, dtype: int64

In [87]:
H10_df = H9_df.groupby(['station_id']).ffill()

In [88]:
H10_df.count()

station_id             102248
time                   102248
station_latitude       102248
station_longitude      102248
                        ...  
num_bikes_available     68408
num_docks_available     68408
capacity                68408
bike_flow_count         68408
Length: 11, dtype: int64

In [89]:
H11_df = H10_df.dropna()
#H11_df = H10_df.copy()

In [90]:
H11_df

,station_id,time,station_latitude,station_longitude,departure_count,arrival_count,trip_flow_count,num_bikes_available,num_docks_available,capacity,bike_flow_count
0,3,2012-09-01 00:00:00,42.340021,-71.100812,0,0,0,5.0,10.0,15.0,0.0
1,3,2012-09-01 01:00:00,42.340021,-71.100812,0,0,0,5.0,10.0,15.0,-3.0
2,3,2012-09-01 02:00:00,42.340021,-71.100812,0,1,1,2.0,13.0,15.0,0.0
3,3,2012-09-01 03:00:00,42.340021,-71.100812,0,0,0,2.0,13.0,15.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
68404,98,2012-09-30 20:00:00,42.371848,-71.060292,1,0,-1,4.0,14.0,18.0,-4.0
68405,98,2012-09-30 21:00:00,42.371848,-71.060292,0,2,2,0.0,18.0,18.0,0.0
68406,98,2012-09-30 22:00:00,42.371848,-71.060292,0,0,0,0.0,18.0,18.0,7.0
68407,98,2012-09-30 23:00:00,42.371848,-71.060292,0,0,0,7.0,11.0,18.0,2.0


In [91]:
H11_df.nunique()

station_id              95
time                   723
station_latitude        95
station_longitude       95
                      ... 
num_bikes_available     48
num_docks_available     48
capacity                28
bike_flow_count         53
Length: 11, dtype: int64

In [92]:
H11_df.set_index(_station_time_list, drop=True, inplace=True)

In [93]:
#H11_df.groupby('station_id').ffill()

In [94]:
H11_df = H11_df.astype('int64')

In [95]:
H11_df

departure_count  \
station_id station_latitude station_longitude time                                   
3          42.340021        -71.100812        2012-09-01 00:00:00                0   
                                              2012-09-01 01:00:00                0   
                                              2012-09-01 02:00:00                0   
                                              2012-09-01 03:00:00                0   
...                                                                            ...   
98         42.371848        -71.060292        2012-09-30 20:00:00                1   
                                              2012-09-30 21:00:00                0   
                                              2012-09-30 22:00:00                0   
                                              2012-09-30 23:00:00                0   

                                                                   arrival_count  \
station_id station_latitude station_longitude time                                 
3          42.340021        -71.100812        2012-09-01 00:00:00              0   
                                              2012-09-01 01:00:00              0   
                                              2012-09-01 02:00:00              1   
                                              2012-09-01 03:00:00              0   
...                                                                          ...   
98         42.371848        -71.060292        2012-09-30 20:00:00              0   
                                              2012-09-30 21:00:00              2   
                                              2012-09-30 22:00:00              0   
                                              2012-09-30 23:00:00              0   

                                                                   trip_flow_count  \
station_id station_latitude station_longitude time                                   
3          42.340021        -71.100812        2012-09-01 00:00:00                0   
                                              2012-09-01 01:00:00                0   
                                              2012-09-01 02:00:00                1   
                                              2012-09-01 03:00:00                0   
...                                                                            ...   
98         42.371848        -71.060292        2012-09-30 20:00:00               -1   
                                              2012-09-30 21:00:00                2   
                                              2012-09-30 22:00:00                0   
                                              2012-09-30 23:00:00                0   

                                                                   num_bikes_available  \
station_id station_latitude station_longitude time                                       
3          42.340021        -71.100812        2012-09-01 00:00:00                    5   
                                              2012-09-01 01:00:00                    5   
                                              2012-09-01 02:00:00                    2   
                                              2012-09-01 03:00:00                    2   
...                                                                                ...   
98         42.371848        -71.060292        2012-09-30 20:00:00                    4   
                                              2012-09-30 21:00:00                    0   
                                              2012-09-30 22:00:00                    0   
                                              2012-09-30 23:00:00                    7   

                                                                   num_docks_available  \
station_id station_latitude station_longitude time                                       
3          42.340021        -71.100812        2012-09-01 00:00:00            

In [96]:
H11_df['rebalancing_flow_count'] = H11_df[['bike_flow_count', 'trip_flow_count']].apply(lambda x: x[0] - x[1], axis=1)

In [97]:
H11_df

departure_count  \
station_id station_latitude station_longitude time                                   
3          42.340021        -71.100812        2012-09-01 00:00:00                0   
                                              2012-09-01 01:00:00                0   
                                              2012-09-01 02:00:00                0   
                                              2012-09-01 03:00:00                0   
...                                                                            ...   
98         42.371848        -71.060292        2012-09-30 20:00:00                1   
                                              2012-09-30 21:00:00                0   
                                              2012-09-30 22:00:00                0   
                                              2012-09-30 23:00:00                0   

                                                                   arrival_count  \
station_id station_latitude station_longitude time                                 
3          42.340021        -71.100812        2012-09-01 00:00:00              0   
                                              2012-09-01 01:00:00              0   
                                              2012-09-01 02:00:00              1   
                                              2012-09-01 03:00:00              0   
...                                                                          ...   
98         42.371848        -71.060292        2012-09-30 20:00:00              0   
                                              2012-09-30 21:00:00              2   
                                              2012-09-30 22:00:00              0   
                                              2012-09-30 23:00:00              0   

                                                                   trip_flow_count  \
station_id station_latitude station_longitude time                                   
3          42.340021        -71.100812        2012-09-01 00:00:00                0   
                                              2012-09-01 01:00:00                0   
                                              2012-09-01 02:00:00                1   
                                              2012-09-01 03:00:00                0   
...                                                                            ...   
98         42.371848        -71.060292        2012-09-30 20:00:00               -1   
                                              2012-09-30 21:00:00                2   
                                              2012-09-30 22:00:00                0   
                                              2012-09-30 23:00:00                0   

                                                                   num_bikes_available  \
station_id station_latitude station_longitude time                                       
3          42.340021        -71.100812        2012-09-01 00:00:00                    5   
                                              2012-09-01 01:00:00                    5   
                                              2012-09-01 02:00:00                    2   
                                              2012-09-01 03:00:00                    2   
...                                                                                ...   
98         42.371848        -71.060292        2012-09-30 20:00:00                    4   
                                              2012-09-30 21:00:00                    0   
                                              2012-09-30 22:00:00                    0   
                                              2012-09-30 23:00:00                    7   

                                                                   num_docks_available  \
station_id station_latitude station_longitude time                                       
3          42.340021        -71.100812        2012-09-01 00:00:00            

In [98]:
H12_df = H11_df.copy()

In [99]:
H12_df['cumulative_trip_flow_count'] = H12_df.groupby('station_id')['trip_flow_count'].cumsum()

In [100]:
H12_df

departure_count  \
station_id station_latitude station_longitude time                                   
3          42.340021        -71.100812        2012-09-01 00:00:00                0   
                                              2012-09-01 01:00:00                0   
                                              2012-09-01 02:00:00                0   
                                              2012-09-01 03:00:00                0   
...                                                                            ...   
98         42.371848        -71.060292        2012-09-30 20:00:00                1   
                                              2012-09-30 21:00:00                0   
                                              2012-09-30 22:00:00                0   
                                              2012-09-30 23:00:00                0   

                                                                   arrival_count  \
station_id station_latitude station_longitude time                                 
3          42.340021        -71.100812        2012-09-01 00:00:00              0   
                                              2012-09-01 01:00:00              0   
                                              2012-09-01 02:00:00              1   
                                              2012-09-01 03:00:00              0   
...                                                                          ...   
98         42.371848        -71.060292        2012-09-30 20:00:00              0   
                                              2012-09-30 21:00:00              2   
                                              2012-09-30 22:00:00              0   
                                              2012-09-30 23:00:00              0   

                                                                   trip_flow_count  \
station_id station_latitude station_longitude time                                   
3          42.340021        -71.100812        2012-09-01 00:00:00                0   
                                              2012-09-01 01:00:00                0   
                                              2012-09-01 02:00:00                1   
                                              2012-09-01 03:00:00                0   
...                                                                            ...   
98         42.371848        -71.060292        2012-09-30 20:00:00               -1   
                                              2012-09-30 21:00:00                2   
                                              2012-09-30 22:00:00                0   
                                              2012-09-30 23:00:00                0   

                                                                   num_bikes_available  \
station_id station_latitude station_longitude time                                       
3          42.340021        -71.100812        2012-09-01 00:00:00                    5   
                                              2012-09-01 01:00:00                    5   
                                              2012-09-01 02:00:00                    2   
                                              2012-09-01 03:00:00                    2   
...                                                                                ...   
98         42.371848        -71.060292        2012-09-30 20:00:00                    4   
                                              2012-09-30 21:00:00                    0   
                                              2012-09-30 22:00:00                    0   
                                              2012-09-30 23:00:00                    7   

                                                                   num_docks_available  \
station_id station_latitude station_longitude time                                       
3          42.340021        -71.100812        2012-09-01 00:00:00            

In [101]:
H12_df.reset_index(inplace=True)
H12_df.set_index(['station_id'], inplace=True)

In [102]:
H12_df

,station_latitude,station_longitude,time,departure_count,arrival_count,trip_flow_count,num_bikes_available,num_docks_available,capacity,bike_flow_count,rebalancing_flow_count,cumulative_trip_flow_count
station_id,,,,,,,,,,,,
3,42.340021,-71.100812,2012-09-01 00:00:00,0,0,0,5,10,15,0,0,0
3,42.340021,-71.100812,2012-09-01 01:00:00,0,0,0,5,10,15,-3,-3,0
3,42.340021,-71.100812,2012-09-01 02:00:00,0,1,1,2,13,15,0,-1,1
3,42.340021,-71.100812,2012-09-01 03:00:00,0,0,0,2,13,15,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
98,42.371848,-71.060292,2012-09-30 20:00:00,1,0,-1,4,14,18,-4,-3,-55
98,42.371848,-71.060292,2012-09-30 21:00:00,0,2,2,0,18,18,0,-2,-53
98,42.371848,-71.060292,2012-09-30 22:00:00,0,0,0,0,18,18,7,7,-53
98,42.371848,-71.060292,2012-09-30 23:00:00,0,0,0,7,11,18,2,2,-53


In [103]:

SS_Init_df = SS4_1_df.query(f'time == "{period_beg}"').set_index(['station_id'])[['num_bikes_available']]\
.rename(columns={'num_bikes_available':'initial_num_bikes_available'})

In [104]:
SS_Init_df

,initial_num_bikes_available
station_id,
3,5
4,14
5,2
6,8
...,...
86,11
88,13
90,7
91,2


In [105]:
H13_df = pd.merge(H12_df, SS_Init_df, how='left', left_index=True, right_index=True)

In [106]:
H13_df

,station_latitude,station_longitude,time,departure_count,arrival_count,trip_flow_count,num_bikes_available,num_docks_available,capacity,bike_flow_count,rebalancing_flow_count,cumulative_trip_flow_count,initial_num_bikes_available
station_id,,,,,,,,,,,,,
3,42.340021,-71.100812,2012-09-01 00:00:00,0,0,0,5,10,15,0,0,0,5.0
3,42.340021,-71.100812,2012-09-01 01:00:00,0,0,0,5,10,15,-3,-3,0,5.0
3,42.340021,-71.100812,2012-09-01 02:00:00,0,1,1,2,13,15,0,-1,1,5.0
3,42.340021,-71.100812,2012-09-01 03:00:00,0,0,0,2,13,15,1,1,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,42.371848,-71.060292,2012-09-30 20:00:00,1,0,-1,4,14,18,-4,-3,-55,NaN
98,42.371848,-71.060292,2012-09-30 21:00:00,0,2,2,0,18,18,0,-2,-53,NaN
98,42.371848,-71.060292,2012-09-30 22:00:00,0,0,0,0,18,18,7,7,-53,NaN
98,42.371848,-71.060292,2012-09-30 23:00:00,0,0,0,7,11,18,2,2,-53,NaN


In [107]:
H13_df['initial_num_bikes_available'].fillna(0, inplace=True)
H13_df = H13_df.astype({'initial_num_bikes_available':'int64'})

In [108]:
H13_df

,station_latitude,station_longitude,time,departure_count,arrival_count,trip_flow_count,num_bikes_available,num_docks_available,capacity,bike_flow_count,rebalancing_flow_count,cumulative_trip_flow_count,initial_num_bikes_available
station_id,,,,,,,,,,,,,
3,42.340021,-71.100812,2012-09-01 00:00:00,0,0,0,5,10,15,0,0,0,5
3,42.340021,-71.100812,2012-09-01 01:00:00,0,0,0,5,10,15,-3,-3,0,5
3,42.340021,-71.100812,2012-09-01 02:00:00,0,1,1,2,13,15,0,-1,1,5
3,42.340021,-71.100812,2012-09-01 03:00:00,0,0,0,2,13,15,1,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,42.371848,-71.060292,2012-09-30 20:00:00,1,0,-1,4,14,18,-4,-3,-55,0
98,42.371848,-71.060292,2012-09-30 21:00:00,0,2,2,0,18,18,0,-2,-53,0
98,42.371848,-71.060292,2012-09-30 22:00:00,0,0,0,0,18,18,7,7,-53,0
98,42.371848,-71.060292,2012-09-30 23:00:00,0,0,0,7,11,18,2,2,-53,0


In [109]:
H13_df['num_bikes_available_without_rebalancing'] \
= H13_df[['cumulative_trip_flow_count', 'initial_num_bikes_available']].apply(lambda x: x[0] + x[1], axis=1)

In [110]:
H13_df.reset_index(inplace=True)

In [111]:
H13_df = H13_df.sort_values(by=['time', 'station_id'])

In [112]:
H13_df.set_index(['time', 'station_id', 'station_latitude', 'station_longitude'], inplace=True)

In [113]:
H13_df

departure_count  \
time                station_id station_latitude station_longitude                    
2012-09-01 00:00:00 3          42.340021        -71.100812                       0   
                    4          42.345392        -71.069616                       0   
                    5          42.341814        -71.090179                       0   
                    6          42.361285        -71.065140                       2   
...                                                                            ...   
2012-10-01 00:00:00 45         42.344763        -71.097880                       0   
                    74         42.373268        -71.118579                       0   
2012-10-01 12:00:00 65         42.347500        -71.044100                       0   
2012-10-01 13:00:00 47         42.362811        -71.056067                       0   

                                                                   arrival_count  \
time                station_id station_latitude station_longitude                  
2012-09-01 00:00:00 3          42.340021        -71.100812                     0   
                    4          42.345392        -71.069616                     0   
                    5          42.341814        -71.090179                     0   
                    6          42.361285        -71.065140                     0   
...                                                                          ...   
2012-10-01 00:00:00 45         42.344763        -71.097880                     2   
                    74         42.373268        -71.118579                     1   
2012-10-01 12:00:00 65         42.347500        -71.044100                     1   
2012-10-01 13:00:00 47         42.362811        -71.056067                     1   

                                                                   trip_flow_count  \
time                station_id station_latitude station_longitude                    
2012-09-01 00:00:00 3          42.340021        -71.100812                       0   
                    4          42.345392        -71.069616                       0   
                    5          42.341814        -71.090179                       0   
                    6          42.361285        -71.065140                      -2   
...                                                                            ...   
2012-10-01 00:00:00 45         42.344763        -71.097880                       2   
                    74         42.373268        -71.118579                       1   
2012-10-01 12:00:00 65         42.347500        -71.044100                       1   
2012-10-01 13:00:00 47         42.362811        -71.056067                       1   

                                                                   num_bikes_available  \
time                station_id station_latitude station_longitude                        
2012-09-01 00:00:00 3          42.340021        -71.100812                           5   
                    4          42.345392        -71.069616                          14   
                    5          42.341814        -71.090179                           2   
                    6          42.361285        -71.065140                           8   
...                                                                                ...   
2012-10-01 00:00:00 45         42.344763        -71.097880                           7   
                    74         42.373268        -71.118579                          10   
2012-10-01 12:00:00 65         42.347500        -71.044100                           5   
2012-10-01 13:00:00 47         42.362811        -71.056067                           2   

                                                                   num_docks_available  \
time                station_id station_latitude station_longitude                        
2012-09-01 00:00:00 3          42.340021        -71.100812                   

In [114]:
H13_df.to_csv(f'Boston_Hubway_Station_Flow_{period_beg[:7]}.csv')

In [115]:
D01_df = H13_df.reset_index().groupby(['time'])[['departure_count', 'arrival_count']].sum()\
.rename(columns={'departure_count':'total_departure_count', 'arrival_count':'total_arrival_count'})

In [116]:
D01_df

,total_departure_count,total_arrival_count
time,,
2012-09-01 00:00:00,64,45
2012-09-01 01:00:00,36,49
2012-09-01 02:00:00,23,22
2012-09-01 03:00:00,5,10
...,...,...
2012-09-30 23:00:00,25,31
2012-10-01 00:00:00,0,7
2012-10-01 12:00:00,0,1
2012-10-01 13:00:00,0,1


In [117]:
D01_df.to_csv(f'Boston_Hubway_Flow_{period_beg[:7]}.csv')

In [118]:
P01_df = pd.merge(H13_df.reset_index().set_index(['time'])\
                  [['station_id', 'station_latitude', 'station_longitude',
                    'capacity', 'initial_num_bikes_available',
                    'departure_count', 'arrival_count']],
                  D01_df, left_index=True, right_index=True)

In [119]:
P01_df

,station_id,station_latitude,station_longitude,capacity,initial_num_bikes_available,departure_count,arrival_count,total_departure_count,total_arrival_count
time,,,,,,,,,
2012-09-01 00:00:00,3,42.340021,-71.100812,15,5,0,0,64,45
2012-09-01 00:00:00,4,42.345392,-71.069616,15,14,0,0,64,45
2012-09-01 00:00:00,5,42.341814,-71.090179,14,2,0,0,64,45
2012-09-01 00:00:00,6,42.361285,-71.065140,15,8,2,0,64,45
...,...,...,...,...,...,...,...,...,...
2012-10-01 00:00:00,45,42.344763,-71.097880,19,16,0,2,0,7
2012-10-01 00:00:00,74,42.373268,-71.118579,17,18,0,1,0,7
2012-10-01 12:00:00,65,42.347500,-71.044100,19,5,0,1,0,1
2012-10-01 13:00:00,47,42.362811,-71.056067,11,6,0,1,0,1


In [120]:
P01_df['departure_relative_frequency'] = P01_df[['departure_count', 'total_departure_count']].apply(lambda x: x[0] / x[1], axis=1)
P01_df['arrival_relative_frequency'] = P01_df[['arrival_count', 'total_arrival_count']].apply(lambda x: x[0] / x[1], axis=1)


C:\Users\U\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in longlong_scalars
  """Entry point for launching an IPython kernel.
C:\Users\U\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in longlong_scalars
  


In [121]:
P01_df = P01_df.fillna(1/95)


In [122]:
P01_df['departure_cumulative_relative_frequency'] = P01_df.groupby('time')['departure_relative_frequency'].cumsum()
P01_df['arrival_cumulative_relative_frequency'] = P01_df.groupby('time')['arrival_relative_frequency'].cumsum()

In [123]:
P01_df

,station_id,station_latitude,station_longitude,capacity,initial_num_bikes_available,departure_count,arrival_count,total_departure_count,total_arrival_count,departure_relative_frequency,arrival_relative_frequency,departure_cumulative_relative_frequency,arrival_cumulative_relative_frequency
time,,,,,,,,,,,,,
2012-09-01 00:00:00,3,42.340021,-71.100812,15,5,0,0,64,45,0.000000,0.000000,0.000000,0.000000
2012-09-01 00:00:00,4,42.345392,-71.069616,15,14,0,0,64,45,0.000000,0.000000,0.000000,0.000000
2012-09-01 00:00:00,5,42.341814,-71.090179,14,2,0,0,64,45,0.000000,0.000000,0.000000,0.000000
2012-09-01 00:00:00,6,42.361285,-71.065140,15,8,2,0,64,45,0.031250,0.000000,0.031250,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-01 00:00:00,45,42.344763,-71.097880,19,16,0,2,0,7,0.010526,0.285714,0.052632,0.857143
2012-10-01 00:00:00,74,42.373268,-71.118579,17,18,0,1,0,7,0.010526,0.142857,0.063158,1.000000
2012-10-01 12:00:00,65,42.347500,-71.044100,19,5,0,1,0,1,0.010526,1.000000,0.010526,1.000000
2012-10-01 13:00:00,47,42.362811,-71.056067,11,6,0,1,0,1,0.010526,1.000000,0.010526,1.000000


In [143]:
P02_df = P01_df[station_list + \
                ['capacity', 'initial_num_bikes_available',
                'total_departure_count', 'total_arrival_count', 
                'departure_cumulative_relative_frequency', 'arrival_cumulative_relative_frequency',
                'departure_count', 'arrival_count']]

In [144]:
P02_df

,station_id,station_latitude,station_longitude,capacity,initial_num_bikes_available,total_departure_count,total_arrival_count,departure_cumulative_relative_frequency,arrival_cumulative_relative_frequency,departure_count,arrival_count
time,,,,,,,,,,,
2012-09-01 00:00:00,3,42.340021,-71.100812,15,5,64,45,0.000000,0.000000,0,0
2012-09-01 00:00:00,4,42.345392,-71.069616,15,14,64,45,0.000000,0.000000,0,0
2012-09-01 00:00:00,5,42.341814,-71.090179,14,2,64,45,0.000000,0.000000,0,0
2012-09-01 00:00:00,6,42.361285,-71.065140,15,8,64,45,0.031250,0.000000,2,0
...,...,...,...,...,...,...,...,...,...,...,...
2012-10-01 00:00:00,45,42.344763,-71.097880,19,16,0,7,0.052632,0.857143,0,2
2012-10-01 00:00:00,74,42.373268,-71.118579,17,18,0,7,0.063158,1.000000,0,1
2012-10-01 12:00:00,65,42.347500,-71.044100,19,5,0,1,0.010526,1.000000,0,1
2012-10-01 13:00:00,47,42.362811,-71.056067,11,6,0,1,0.010526,1.000000,0,1


In [145]:
P02_df.to_csv(f'Boston_Hubway_Station_CRF_{period_beg[:7]}.csv')

In [146]:
S01_df = H13_df.reset_index()[station_list].drop_duplicates()
S01_df

,station_id,station_latitude,station_longitude
0,3,42.340021,-71.100812
1,4,42.345392,-71.069616
2,5,42.341814,-71.090179
3,6,42.361285,-71.065140
...,...,...,...
91,95,42.372969,-71.094445
92,96,42.373379,-71.111075
93,97,42.369182,-71.117152
94,98,42.371848,-71.060292


In [147]:
S01_df['key'] = 0

In [148]:
S01_df

,station_id,station_latitude,station_longitude,key
0,3,42.340021,-71.100812,0
1,4,42.345392,-71.069616,0
2,5,42.341814,-71.090179,0
3,6,42.361285,-71.065140,0
...,...,...,...,...
91,95,42.372969,-71.094445,0
92,96,42.373379,-71.111075,0
93,97,42.369182,-71.117152,0
94,98,42.371848,-71.060292,0


In [149]:
S02_df = pd.merge(S01_df, S01_df, on='key', suffixes=['_source', '_target']).drop('key', axis=1)

In [150]:
S02_df

,station_id_source,station_latitude_source,station_longitude_source,station_id_target,station_latitude_target,station_longitude_target
0,3,42.340021,-71.100812,3,42.340021,-71.100812
1,3,42.340021,-71.100812,4,42.345392,-71.069616
2,3,42.340021,-71.100812,5,42.341814,-71.090179
3,3,42.340021,-71.100812,6,42.361285,-71.065140
...,...,...,...,...,...,...
9021,98,42.371848,-71.060292,95,42.372969,-71.094445
9022,98,42.371848,-71.060292,96,42.373379,-71.111075
9023,98,42.371848,-71.060292,97,42.369182,-71.117152
9024,98,42.371848,-71.060292,98,42.371848,-71.060292


In [151]:
S02_df.to_csv(f'Boston_Hubway_Stations_Cross_Join.csv')

In [152]:
from geopy import distance

In [153]:
S03_df = S02_df.query('station_id_source != station_id_target').copy()

In [154]:
S03_df['interstation_distance'] = S03_df.apply(lambda x: distance.distance((x[1], x[2]), (x[4], x[5])).km, axis=1)

In [155]:
S03_df

,station_id_source,station_latitude_source,station_longitude_source,station_id_target,station_latitude_target,station_longitude_target,interstation_distance
1,3,42.340021,-71.100812,4,42.345392,-71.069616,2.639021
2,3,42.340021,-71.100812,5,42.341814,-71.090179,0.898584
3,3,42.340021,-71.100812,6,42.361285,-71.065140,3.770650
4,3,42.340021,-71.100812,7,42.353412,-71.044624,4.862941
...,...,...,...,...,...,...,...
9020,98,42.371848,-71.060292,94,42.375603,-71.064608,0.548040
9021,98,42.371848,-71.060292,95,42.372969,-71.094445,2.815798
9022,98,42.371848,-71.060292,96,42.373379,-71.111075,4.186231
9023,98,42.371848,-71.060292,97,42.369182,-71.117152,4.692821


In [156]:
S04_df = S03_df.sort_values(by=['station_id_source', 'interstation_distance'])[['station_id_source', 'station_id_target', 'interstation_distance']]

In [157]:
S04_df

,station_id_source,station_id_target,interstation_distance
11,3,14,0.356344
28,3,32,0.447536
8,3,11,0.493587
41,3,45,0.579511
...,...,...,...
8992,98,66,6.598967
8935,98,8,6.669860
9018,98,92,6.969045
8963,98,37,7.280486


In [158]:
S04_df['interstation_distance_rank'] = S04_df.groupby(['station_id_source'])['interstation_distance'].rank(method='first').apply(lambda x: int(x - 1))

In [159]:
S04_df.set_index('station_id_source', inplace=True)

In [160]:
S04_df

,station_id_target,interstation_distance,interstation_distance_rank
station_id_source,,,
3,14,0.356344,0
3,32,0.447536,1
3,11,0.493587,2
3,45,0.579511,3
...,...,...,...
98,66,6.598967,90
98,8,6.669860,91
98,92,6.969045,92
98,37,7.280486,93


In [161]:
S04_df.to_csv(f'Boston_Hubway_Interstation_Distance.csv')